In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np

from utils import (read_single_problem_from_path_as_adjacency,
 read_single_problem_from_path_as_sparse, 
 read_single_problem_from_path_as_sparse_from_adjacency,
 order_nodes_in_descending_order)
from constants import BASE_INSTANCES_FILES
from heuristics import descending_degree_glutonous_heuristic

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DR = INSTANCES_DIR / "other_instances"

In [3]:
_,_, graph, degrees = read_single_problem_from_path_as_adjacency(
        instance_path=BASE_INSTANCES_DIR / "random-10.col"
    )
print(graph)
descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)

[[0 1 0 1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 0 1]
 [1 0 1 0 0 0 0 0 1 0]
 [1 1 0 0 0 1 1 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 0 1 1 0 1 1 1]
 [0 1 0 0 1 0 1 0 0 1]
 [1 1 0 1 0 1 1 0 0 0]
 [0 1 1 0 1 0 1 1 0 0]]


[4, 6, 5]